[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/rag-chatbot.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/rag-chatbot.ipynb)

# Building RAG Chatbots with LangChain

In this example, we'll work on building an AI chatbot from start-to-finish. We will be using LangChain, OpenAI, and Pinecone vector DB, to build a chatbot capable of learning from the external world using **R**etrieval **A**ugmented **G**eneration (RAG).

We will be using a dataset sourced from the Llama 2 ArXiv paper and other related papers to help our chatbot answer questions about the latest and greatest in the world of GenAI.

By the end of the example we'll have a functioning chatbot and RAG pipeline that can hold a conversation and provide informative responses based on a knowledge base.

### Before you begin

You'll need to get an [OpenAI API key](https://platform.openai.com/account/api-keys) and [Pinecone API key](https://app.pinecone.io).

### Prerequisites

Before we start building our chatbot, we need to install some Python libraries. Here's a brief overview of what each library does:

- **langchain**: This is a library for GenAI. We'll use it to chain together different language models and components for our chatbot.
- **openai**: This is the official OpenAI Python client. We'll use it to interact with the OpenAI API and generate responses for our chatbot.
- **datasets**: This library provides a vast array of datasets for machine learning. We'll use it to load our knowledge base for the chatbot.
- **pinecone-client**: This is the official Pinecone Python client. We'll use it to interact with the Pinecone API and store our chatbot's knowledge base in a vector database.

You can install these libraries using pip like so:

In [2]:
%pip install -qU \
    langchain==0.0.292 \
    openai==0.28.0 \
    datasets==2.10.1 \
    pinecone-client==2.2.4 \
    tiktoken==0.5.1

  error: subprocess-exited-with-error
  
  × Building wheel for tiktoken (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [38 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-312
      creating build/lib.linux-x86_64-cpython-312/tiktoken
      copying tiktoken/registry.py -> build/lib.linux-x86_64-cpython-312/tiktoken
      copying tiktoken/core.py -> build/lib.linux-x86_64-cpython-312/tiktoken
      copying tiktoken/load.py -> build/lib.linux-x86_64-cpython-312/tiktoken
      copying tiktoken/model.py -> build/lib.linux-x86_64-cpython-312/tiktoken
      copying tiktoken/_educational.py -> build/lib.linux-x86_64-cpython-312/tiktoken
      copying tiktoken/__init__.py -> build/lib.linux-x86_64-cpython-312/tiktoken
      creating build/lib.linux-x86_64-cpython-312/tiktoken_ext
      copying tiktoken_ext/openai_public.py -> build/lib.linux-x86_64-cpython-312/tiktoken_ex

In [64]:
import importlib

libraries = [
    "langchain",
    "openai",
    "datasets",
    "pinecone_client",
    "tiktoken"
]

for library in libraries:
    try:
        module = importlib.import_module(library)
        print(f"{library} is installed (version: {module.__version__})")
    except ModuleNotFoundError:
        print(f"{library} is not installed")

langchain is installed (version: 0.0.292)
openai is installed (version: 0.28.0)
datasets is installed (version: 2.10.1)
pinecone_client is not installed
tiktoken is not installed


In [65]:
%pip install pinecone-client==2.2.4
%pip install tiktoken==0.5.1

Note: you may need to restart the kernel to use updated packages.
  Using cached tiktoken-0.5.1.tar.gz (32 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached regex-2023.12.25-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
Using cached regex-2023.12.25-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (789 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for tiktoken (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [38 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-312
      creating build/lib.linux-x86_64-cpython-312/tiktoken
      copying tiktoken/registry.py -> build/lib.linux-x86_64-cpython-312/tiktoken
      copying tiktoken/core.py -> build/lib.linux-x86_64-cpython-312/tiktoken
      copying tiktoken/load.p

### Building a Chatbot (no RAG)

We will be relying heavily on the LangChain library to bring together the different components needed for our chatbot. To begin, we'll create a simple chatbot without any retrieval augmentation. We do this by initializing a `ChatOpenAI` object. For this we do need an [OpenAI API key](https://platform.openai.com/account/api-keys).

In [2]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY") 
os.environ["OPENAI_API_KEY"] = openai_api_key 

# print(OPENAI_API_KEY)
chat = ChatOpenAI(
    openai_api_key = openai_api_key,
    model='gpt-3.5-turbo'
)

Chats with OpenAI's `gpt-3.5-turbo` and `gpt-4` chat models are typically structured (in plain text) like this:

```
System: You are a helpful assistant.

User: Hi AI, how are you today?

Assistant: I'm great thank you. How can I help you?

User: I'd like to understand string theory.

Assistant:
```

The final `"Assistant:"` without a response is what would prompt the model to continue the conversation. In the official OpenAI `ChatCompletion` endpoint these would be passed to the model in a format like:

```python
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi AI, how are you today?"},
    {"role": "assistant", "content": "I'm great thank you. How can I help you?"}
    {"role": "user", "content": "I'd like to understand string theory."}
]
```

In LangChain there is a slightly different format. We use three _message_ objects like so:

In [29]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

The format is very similar, we're just swapped the role of `"user"` for `HumanMessage`, and the role of `"assistant"` for `AIMessage`.

We generate the next response from the AI by passing these messages to the `ChatOpenAI` object.

In [30]:
res = chat(messages)
res

AIMessage(content='String theory is a theoretical framework in physics that attempts to explain the fundamental nature of particles and forces in the universe. According to string theory, particles are not considered as point-like objects but rather as tiny, vibrating strings. These strings can vibrate at different frequencies, and each frequency corresponds to a different particle.\n\nHere are some key points to understand about string theory:\n\n1. Vibrating Strings: In string theory, particles are not represented as zero-dimensional points but as one-dimensional strings. The vibrations of these strings determine the properties and behavior of particles.\n\n2. Extra Dimensions: String theory suggests that the universe has additional spatial dimensions beyond the three we are familiar with (length, width, and height). These extra dimensions are typically compactified or "curled up" at extremely small scales, making them difficult to detect.\n\n3. Unification of Forces: One of the goal

In response we get another AI message object. We can print it more clearly like so:

In [31]:
print(res.content)

String theory is a theoretical framework in physics that attempts to explain the fundamental nature of particles and forces in the universe. According to string theory, particles are not considered as point-like objects but rather as tiny, vibrating strings. These strings can vibrate at different frequencies, and each frequency corresponds to a different particle.

Here are some key points to understand about string theory:

1. Vibrating Strings: In string theory, particles are not represented as zero-dimensional points but as one-dimensional strings. The vibrations of these strings determine the properties and behavior of particles.

2. Extra Dimensions: String theory suggests that the universe has additional spatial dimensions beyond the three we are familiar with (length, width, and height). These extra dimensions are typically compactified or "curled up" at extremely small scales, making them difficult to detect.

3. Unification of Forces: One of the goals of string theory is to un

Because `res` is just another `AIMessage` object, we can append it to `messages`, add another `HumanMessage`, and generate the next response in the conversation.

In [32]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do physicists believe it can produce a 'unified theory'?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Physicists believe that string theory has the potential to produce a unified theory because it provides a framework that can encompass all the fundamental forces of nature, including gravity, electromagnetism, and the strong and weak nuclear forces. Here are a few reasons why physicists believe string theory has the potential for unification:

1. Consistency of Mathematics: String theory is based on a consistent mathematical framework that allows for the incorporation of quantum mechanics and general relativity. This is important because these two theories are currently described by separate mathematical frameworks that are not easily reconciled.

2. Gravity and Quantum Mechanics: One of the key challenges in physics is combining gravity, described by general relativity, with quantum mechanics. In string theory, the graviton, which is the hypothetical particle responsible for transmitting the force of gravity, naturally emerges from the vibrational modes of strings. String theory provi

### Dealing with Hallucinations

We have our chatbot, but as mentioned — the knowledge of LLMs can be limited. The reason for this is that LLMs learn all they know during training. An LLM essentially compresses the "world" as seen in the training data into the internal parameters of the model. We call this knowledge the _parametric knowledge_ of the model.

By default, LLMs have no access to the external world.

The result of this is very clear when we ask LLMs about more recent information, like about the new (and very popular) Llama 2 LLM.

In [41]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What is so special about Llama 2?"
)

# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [42]:
print(res.content)

I apologize, but I couldn't find any specific information about "Llama 2". It's possible that it may refer to something specific within a particular context or industry. Could you please provide more details or clarify your query so that I can assist you better?


Our chatbot can no longer help us, it doesn't contain the information we need to answer the question. It was very clear from this answer that the LLM doesn't know the informaiton, but sometimes an LLM may respond like it _does_ know the answer — and this can be very hard to detect.

OpenAI have since adjusted the behavior for this particular example as we can see below:

In [43]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Can you tell me about the LLMChain in LangChain?"
)

# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [44]:
print(res.content)

I apologize for the confusion in my previous response. As an AI language model, I don't have specific knowledge about the details of the "LLMChain" in the context of LangChain. It seems that "LLMChain" may be a term specific to the LangChain framework or a specialized usage within a particular domain.

If you have any specific questions about the LangChain framework or any other topic, I'll be happy to help to the best of my abilities.


There is another way of feeding knowledge into LLMs. It is called _source knowledge_ and it refers to any information fed into the LLM via the prompt. We can try that with the LLMChain question. We can take a description of this object from the LangChain documentation.

In [33]:
llmchain_information = [
    "A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.",
    "Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.",
    "LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of data, (2) Be agentic: Allow a language model to interact with its environment. As such, the LangChain framework is designed with the objective in mind to enable those types of applications."
]

source_knowledge = "\n".join(llmchain_information)

We can feed this additional knowledge into our prompt with some instructions telling the LLM how we'd like it to use this information alongside our original query.

In [34]:
query = "Can you tell me about the LLMChain in LangChain?"

augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""

In [35]:
print(augmented_prompt)

Using the contexts below, answer the query.

Contexts:
A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.
Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.
LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of data, (2) Be agentic: Allow a language model to interact with its environment. As such, the LangChain framework is

Now we feed this into our chatbot as we were before.

In [36]:
# create a new user prompt
prompt = HumanMessage(
    content=augmented_prompt
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [37]:
print(res.content)

Certainly! In the context of LangChain, the LLMChain refers to a specific type of chain used within the framework for developing applications powered by language models.

The LLMChain is composed of several modular components, including a PromptTemplate, a model (which can be either an LLM or a ChatModel), and an optional output parser. These components work together to process input variables and generate formatted prompts, which are then passed to the language model for processing.

First, the PromptTemplate takes the input variables and formats them into a prompt. This prompt is designed to provide context and structure for the language model's response.

Next, the model (either an LLM or a ChatModel) receives the prompt and generates a response based on the provided input. The specific model used depends on the requirements of the application.

Finally, the LLMChain may utilize an optional output parser, which is responsible for parsing and formatting the output generated by the la

The quality of this answer is phenomenal. This is made possible thanks to the idea of augmented our query with external knowledge (source knowledge). There's just one problem — how do we get this information in the first place?

We learned in the previous chapters about Pinecone and vector databases. Well, they can help us here too. But first, we'll need a dataset.

### Importing the Data

In this task, we will be importing our data. We will be using the Hugging Face Datasets library to load our data. Specifically, we will be using the `"jamescalam/llama-2-arxiv-papers"` dataset. This dataset contains a collection of ArXiv papers which will serve as the external knowledge base for our chatbot.

In [91]:
from datasets import load_dataset

dataset = load_dataset("bzantium/LITM")

ValueError: Invalid pattern: '**' can only be an entire path component

In [92]:
from datasets import load_dataset

dataset = load_dataset("jamescalam/llama-2-arxiv-papers-chunked")

dataset

ValueError: Invalid pattern: '**' can only be an entire path component

In [93]:
dataset[0]

NameError: name 'dataset' is not defined

#### Dataset Overview

The dataset we are using is sourced from the Llama 2 ArXiv papers. It is a collection of academic papers from ArXiv, a repository of electronic preprints approved for publication after moderation. Each entry in the dataset represents a "chunk" of text from these papers.

Because most **L**arge **L**anguage **M**odels (LLMs) only contain knowledge of the world as it was during training, they cannot answer our questions about Llama 2 — at least not without this data.

### Task 4: Building the Knowledge Base

We now have a dataset that can serve as our chatbot knowledge base. Our next task is to transform that dataset into the knowledge base that our chatbot can use. To do this we must use an embedding model and vector database.

We begin by initializing our connection to Pinecone, this requires a [free API key](https://app.pinecone.io).

In [3]:
import pinecone

# get API key from app.pinecone.io and environment from console
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENVIRONMENT') or 'gcp-starter'
)

Then we initialize the index. We will be using OpenAI's `text-embedding-ada-002` model for creating the embeddings, so we set the `dimension` to `1536`.

In [4]:
import time

index_name = 'llama-2-rag'

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

index = pinecone.Index(index_name)

Then we connect to the index:

In [5]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.04838,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}

Our index is now ready but it's empty. It is a vector index, so it needs vectors. As mentioned, to create these vector embeddings we will OpenAI's `text-embedding-ada-002` model — we can access it via LangChain like so:

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

Using this model we can create embeddings like so:

In [9]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed_model.embed_documents(texts)
len(res), len(res[0])

(2, 1536)

From this we get two (aligning to our two chunks of text) 1536-dimensional embeddings.

We're now ready to embed and index all our our data! We do this by looping through our dataset and embedding and inserting everything in batches.

In [ ]:
from tqdm.auto import tqdm  # for progress bar

data = dataset.to_pandas()  # this makes it easier to iterate over the dataset

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i+batch_size)
    # get batch of data
    batch = data.iloc[i:i_end]
    # generate unique ids for each chunk
    ids = [f"{x['doi']}-{x['chunk-id']}" for i, x in batch.iterrows()]
    # get text to embed
    texts = [x['chunk'] for _, x in batch.iterrows()]
    # embed text
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['chunk'],
         'source': x['source'],
         'title': x['title']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

We can check that the vector index has been populated using `describe_index_stats` like before:

In [25]:
index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.04838,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}

#### Retrieval Augmented Generation

We've built a fully-fledged knowledge base. Now it's time to connect that knowledge base to our chatbot. To do that we'll be diving back into LangChain and reusing our template prompt from earlier.

To use LangChain here we need to load the LangChain abstraction for a vector index, called a `vectorstore`. We pass in our vector `index` to initialize the object.

In [26]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

[01/16/24 09:54:13] WARNING  c:\Users\ekru\AppData\Local\Programs\Python\Python311\Lib\site-package ]8;id=30771;file://c:\Users\ekru\AppData\Local\Programs\Python\Python311\Lib\warnings.py\warnings.py]8;;\:]8;id=24658;file://c:\Users\ekru\AppData\Local\Programs\Python\Python311\Lib\warnings.py#109\109]8;;\
                             s\langchain\vectorstores\pinecone.py:59: UserWarning: Passing in                      
                             `embedding` as a Callable is deprecated. Please pass in an Embeddings                 
                             object instead.                                                                       
                               warnings.warn(                                                                      
                                                                                                                   

Using this `vectorstore` we can already query the index and see if we have any relevant information given our question about Llama 2.

In [27]:
query = "What is so special about Llama 2?"

vectorstore.similarity_search(query, k=3)


[
    Document(
        page_content='Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang\nRoss Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang\nAngela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic\nSergey Edunov Thomas Scialom\x03\nGenAI, Meta\nAbstract\nIn this work, we develop and release Llama 2, a collection of pretrained and ﬁne-tuned\nlarge language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.\nOur ﬁne-tuned LLMs, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc , are optimized for dialogue use cases. Our\nmodels outperform open-source chat models on most benchmarks we tested, and based on\nourhumanevaluationsforhelpfulnessandsafety,maybeasuitablesubstituteforclosedsource models. We provide a detailed description of our approach to ﬁne-tuning and safety',
        metadata={
            'source': 'http://arxiv.org/pdf/2307.09288',
           

We return a lot of text here and it's not that clear what we need or what is relevant. Fortunately, our LLM will be able to parse this information much faster than us. All we need is to connect the output from our `vectorstore` to our `chat` chatbot. To do that we can use the same logic as we used earlier.

In [28]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

Using this we produce an augmented prompt:

In [30]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Edunov Thomas Scialom
GenAI, Meta
Abstract
In this work, we develop and release Llama 2, a collection of pretrained and ﬁne-tuned
large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.
Our ﬁne-tuned LLMs, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc , are optimized for dialogue use cases. Our
models outperform open-source chat models on most benchmarks we tested, and based on
ourhumanevaluationsforhelpfulnessandsafety,maybeasuitablesubstituteforclosedsource models. We provide a detailed description of our approach to ﬁne-tuning and safety
asChatGPT,BARD,andClaude. TheseclosedproductLLMsareheavilyﬁne-tunedtoalignwith

There is still a lot of text here, so let's pass it onto our chat model to see how it performs.

In [32]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

Llama 2 is a collection of pretrained and fine-tuned large language models (LLMs) that are optimized for dialogue use cases. These models range in scale from 7 billion to 70 billion parameters, making them quite large and powerful.

What sets Llama 2 apart is that its fine-tuned LLMs, specifically the L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc models, outperform open-source chat models on most benchmarks tested. Additionally, based on human evaluations for helpfulness and safety, Llama 2 models may serve as suitable substitutes for closed-source models.

The Llama 2 models have undergone a detailed approach to fine-tuning and safety, similar to other closed product LLMs like ChatGPT, BARD, and Claude. These closed product LLMs are heavily fine-tuned to align with human preferences, enhancing their usability and safety.

The performance of Llama 2 models on helpfulness and safety benchmarks generally surpasses existing open-source models. In fact, they appear to be on par w

We can continue with more Llama 2 questions. Let's try _without_ RAG first:

In [33]:
prompt = HumanMessage(
    content="what safety measures were used in the development of llama 2?"
)

res = chat(messages + [prompt])
print(res.content)

In the provided context, it is mentioned that safety measures were taken in the development of Llama 2. However, the specific details regarding the safety measures used are not mentioned. To obtain more information about the safety measures employed in the development of Llama 2, it would be necessary to refer to the original research paper or publication associated with Llama 2.


The chatbot is able to respond about Llama 2 thanks to it's conversational history stored in `messages`. However, it doesn't know anything about the safety measures themselves as we have not provided it with that information via the RAG pipeline. Let's try again but with RAG.

In [34]:
prompt = HumanMessage(
    content=augment_prompt(
        "what safety measures were used in the development of llama 2?"
    )
)

res = chat(messages + [prompt])
print(res.content)

In the development of Llama 2, several safety measures were employed to enhance the safety of the models. These measures include safety-specific data annotation and tuning, red-teaming, iterative evaluations, and a thorough approach to improving LLM safety.

Safety-specific data annotation and tuning involve carefully curating the training data and adjusting the model parameters to prioritize safety and minimize potential harmful outputs.

Red-teaming refers to the process of having external experts assess the safety and potential risks associated with the models. This helps identify any vulnerabilities or issues that may have been overlooked during development.

Iterative evaluations involve continuously assessing the model's performance and making improvements to enhance its safety. This iterative process helps address any shortcomings or concerns that arise during the evaluation phase.

The paper also emphasizes the importance of transparency and openness in sharing the fine-tuning 

We get a much more informed response that includes several items missing in the previous non-RAG response, such as "red-teaming", "iterative evaluations", and the intention of the researchers to share this research to help "improve their safety, promoting responsible development in the field".